In [37]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import math

In [4]:
weights_directory = "Weights/"

num_truck_brand = 9

dataset_directory = "Detected_Data/Brand/"

optimizer_name = 'adam'

In [5]:
batch_size = 4
image_height = 128
image_width = 128

In [6]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(128,128,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

In [7]:
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(rate=0.35))
model.add(Dense(num_truck_brand+1,activation='softmax'))

model.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 128)       147584    
____

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer_name, metrics=['accuracy'])


model_name = weights_directory+optimizer_name+"_TruckBrand_Model_BrandOnly"

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint(model_name+'.h5', save_best_only=True, monitor='val_loss', verbose=1, mode='auto')

In [9]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator()

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        dataset_directory,  # this is the target directory
        target_size=(image_height, image_width),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        dataset_directory,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical')


Found 889 images belonging to 10 classes.
Found 889 images belonging to 10 classes.


In [19]:
model.fit_generator(
        train_generator,
        steps_per_epoch=800 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=20 // batch_size,
        callbacks=[early_stopping, mcp_save, TQDMNotebookCallback()])


Epoch 1/50


200/200 [==============================] - 440s 2s/step - loss: 1.7126 - acc: 0.8937 - val_loss: 1.6118 - val_acc: 0.9000

Epoch 00001: val_loss improved from inf to 1.61181, saving model to Weights/adam_TruckBrand_Model_BrandOnly.h5
Epoch 2/50


200/200 [==============================] - 447s 2s/step - loss: 2.0478 - acc: 0.8637 - val_loss: 0.3198 - val_acc: 0.9500

Epoch 00002: val_loss improved from 1.61181 to 0.31982, saving model to Weights/adam_TruckBrand_Model_BrandOnly.h5
Epoch 3/50


200/200 [==============================] - 480s 2s/step - loss: 0.5639 - acc: 0.9112 - val_loss: 0.4361 - val_acc: 0.9000

Epoch 00003: val_loss did not improve from 0.31982
Epoch 4/50


200/200 [==============================] - 482s 2s/step - loss: 0.5007 - acc: 0.9150 - val_loss: 0.7011 - val_acc: 0.8500

Epoch 00004: val_loss did not improve from 0.31982
Epoch 5/50


200/200 [==============================] - 445s 2s/step - loss: 0.5471 - acc: 0.9075 - val_loss: 0.5265 - val_acc: 0.9000

Epoch 00005: val_loss did not improve from 0.31982
Epoch 6/50


200/200 [==============================] - 445s 2s/step - loss: 0.4931 - acc: 0.9125 - val_loss: 0.5612 - val_acc: 0.9000

Epoch 00006: val_loss did not improve from 0.31982
Epoch 7/50


200/200 [==============================] - 434s 2s/step - loss: 0.5221 - acc: 0.9087 - val_loss: 0.7187 - val_acc: 0.8500

Epoch 00007: val_loss did not improve from 0.31982
Epoch 00007: early stopping



In [24]:
batch_size = 16

generator = test_datagen.flow_from_directory(
            dataset_directory,
            target_size=(image_height, image_width),
            batch_size=batch_size,
            class_mode=None,  # this means our generator will only yield batches of data, no labels
            shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data



Found 889 images belonging to 10 classes.


In [25]:
from keras.models import load_model

loaded_model = load_model(weights_directory+optimizer_name+"_TruckBrand_Model_BrandOnly.h5")

In [26]:
bottleneck_features_train = loaded_model.predict_generator(generator)


In [27]:
score = loaded_model.evaluate_generator(validation_generator)

print("Loss: "+str(score[0]))
print("Accuracy: "+str(score[1]))

Loss: 0.6849432276714806
Accuracy: 0.9111361079865017


In [28]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
import xml.etree.ElementTree as ET
import numpy as np
#import plaidml.keras
import PIL.Image as Image
#plaidml.keras.install_backend()

/opt/conda/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [29]:
from keras.models import load_model


image_height=128
image_width=128

weights_directory = "Weights/"
model = load_model(weights_directory+"adam_TruckBrand_Model_BrandOnly.h5")

In [30]:
def non_max_suppression_fast(boxes, overlapThresh,probs):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

    # initialize the list of picked indexes
    pick = []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    # compute the area of the bounding boxes and sort the bounding
    # boxes by the scores
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(probs)

    # keep looping while some indexes still remain in the indexes
    # list
    
    final_boxes = []
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]

        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
        np.where(overlap > overlapThresh)[0])))

        # return only the bounding boxes that were picked using the
        final_boxes.append(boxes[pick])
    return boxes[pick]

In [49]:
def rcnn(image):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    results = ss.process()
    copy = image.copy()
    copy2 = image.copy()
    positive_boxes = []
    probs = []
    num_boxes = 0
    
    for box in results:
        x1 = box[0]
        y1 = box[1]
        x2 = box[0]+box[2]
        y2 = box[1]+box[3]
        
        roi = image.copy()[y1:y2,x1:x2]       
        roi = cv2.resize(roi,(image_height, image_width))
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi = roi * 1/255.0

        roi_use = roi.reshape((1,image_height, image_width, 3))
        class_pred = model.predict_classes(roi_use)
        
        cv2.imshow("frame", roi_use[0])
        cv2.waitKey(1000)
        prob_type = model.predict(roi_use)[0].argmax()
        prob_value = model.predict(roi_use)[0].max()        
        if prob_type!=8 and prob_value:
            print(prob_type)
            print(prob_value)
            positive_boxes.append([x1,y1,x2,y2])
            probs.append(prob_value)
            cv2.rectangle(copy2,(x1,y1),(x2,y2),(255,0,0),5)
            
        num_boxes+=1
        
        if num_boxes > 1000:
            break
    
    if probs == []:
        return  
    
    cleaned_boxes = non_max_suppression_fast(np.array(positive_boxes),0.1,probs)
    total_boxes = 0
    for clean_box in cleaned_boxes:
        clean_x1 = clean_box[0]
        clean_y1 = clean_box[1]
        clean_x2 = clean_box[2]
        clean_y2 = clean_box[3]
        total_boxes+=1
        cv2.rectangle(copy,(clean_x1,clean_y1),(clean_x2,clean_y2),(0,255,0),3)
    #plt.imshow(copy2)
    plt.imshow(copy)

In [ ]:
test_img = cv2.imread('test/1.jpg')
plt.imshow(test_img)
rcnn(image=test_img)